In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.utils._testing import ignore_warnings
from sklearn import svm
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split
import random

In [9]:
# Read the CSV file
df = pd.read_csv("../data/Occupancy_Estimation.csv")

# Display the first few rows of the DataFrame
df

# df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])


,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124,2018/01/11,08:58:07,25.06,25.13,24.69,25.31,6,7,33,22,0.09,0.04,0.06,0.08,345,0.000000,0,0,0
10125,2018/01/11,08:58:37,25.06,25.06,24.69,25.25,6,7,34,22,0.07,0.05,0.05,0.08,345,0.000000,0,0,0
10126,2018/01/11,08:59:08,25.13,25.06,24.69,25.25,6,7,34,22,0.11,0.05,0.06,0.08,345,0.000000,0,0,0
10127,2018/01/11,08:59:39,25.13,25.06,24.69,25.25,6,7,34,22,0.08,0.08,0.10,0.08,345,0.000000,0,0,0


In [10]:
# Compute cross-validation score
@ignore_warnings(category=ConvergenceWarning)

# Add documetnation to the function
def Kfold_CV_score(df, features, hyperparams=(1), num_folds=5):
    # TODO add doc string

    fold_size = len(df) // num_folds

    fold_train_scores = []
    fold_valid_scores = []

    regularizer = hyperparams

    for i in range(num_folds):
        start = i * fold_size
        end = start + fold_size

        # Extract the training and validation df for this fold 
        valid_data = df[start:end]
        train_data = df[~df.index.isin(range(start, end))]

        X_train = train_data[features]
        Y_train = train_data['Room_Occupancy_Count']

        X_valid = valid_data[features]
        Y_valid = valid_data['Room_Occupancy_Count']

        # Instantiate and train the model on the training data
        mod = svm.SVC(kernel='linear', C=1)

        mod.fit(X_train, Y_train)

        train_score = mod.score(X_train, Y_train)
        valid_score = mod.score(X_valid, Y_valid)

        fold_train_scores.append(train_score)
        fold_valid_scores.append(valid_score)

        # print(f"Fold {i} score: {score}")

    # Calculate the mean score across all folds
    mean_train_score = sum(fold_train_scores) / len(fold_train_scores)
    mean_valid_score = sum(fold_valid_scores) / len(fold_valid_scores)

    return mean_train_score, mean_valid_score

In [11]:

#fit the model to the training data
features = [ 'S1_Temp', 'S2_Temp', 'S3_Temp', 'S4_Temp', 'S1_Light',
            'S2_Light', 'S3_Light', 'S4_Light', 'S1_Sound', 'S2_Sound', 'S3_Sound',
            'S4_Sound', 'S5_CO2', 'S5_CO2_Slope', 'S6_PIR', 'S7_PIR' ]

regularizers = []
hyperparams = (1)
num_folds = 5

# Shuffle the datai
df = df.sample(frac=1)

mean_train_score, mean_valid_score = Kfold_CV_score(
    df=df, features=features, hyperparams=hyperparams, num_folds=num_folds)

print(f"MODEL ARCH ==============================")
print(f"Features: {features}")
print(f"Regularizer: {hyperparams}")
print()
print(f"MODEL EVAL ==============================")
print(f"Num CV folds: {num_folds}")
print(f"Mean train score: {mean_train_score}")
print(f"Mean valid score: {mean_valid_score}")

MODEL ARCH ==============================
Features: ['S1_Temp', 'S2_Temp', 'S3_Temp', 'S4_Temp', 'S1_Light', 'S2_Light', 'S3_Light', 'S4_Light', 'S1_Sound', 'S2_Sound', 'S3_Sound', 'S4_Sound', 'S5_CO2', 'S5_CO2_Slope', 'S6_PIR', 'S7_PIR']
Regularizer: 1

MODEL EVAL ==============================
Num CV folds: 5
Mean train score: 0.9959032576505429
Mean valid score: 0.9890370370370369
